Импорт нужных либ

In [93]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
from sympy import *
from prettytable import PrettyTable

# Задание 2. Вариант 6
Решается обыкновенное дифференциальное уравнение
$ U'''(x) = f(x, U, U', U''), x \in [a; b] $
С краевыми (граничными) условиями
$ U'(a) = A; $
$ U(b) = B, U''(b) = C $
Метод решения уравнения $ \varphi(\alpha) = 0 $ - метод секущих
Направление интегрирования: $ a \implies b $

In [94]:
# Границы отрезка, на котором ищется решение:
a, b = -1, 1
# Число отрезков разбиения
N = 50
# Точность решения нелинейного уравнения
epsilon = 1e-11
# Предельное число итераций
K = 1000
# Начальное значение пристрелочного параметра
alpha0, alpha1 = -1, 1
# Константы краевых уравнений
A, B, C = -2, 1, 2

Данные для работы программы

In [95]:
# Шаг
h = (b - a) / N
iterations_count = 0  # Для итераций в методе секущих

x = Symbol('x')
u = Symbol('u')
v = Symbol('v')
w = Symbol('w')
# Задание функции
# f_exact_solution = x**4 + 2 * u**3 + 3 * v**2 + 4 * w
f_exact_solution = x**3
exact_solution = lambdify([x, u, v, w], f_exact_solution)
# Первая производная
fd_exact_solution = diff(f_exact_solution, x)
derivative_exact_solution = lambdify([x, u, v, w], fd_exact_solution)
# Вторая производная
f2d_exact_solution = diff(fd_exact_solution, )
derivative2_exact_solution = lambdify([x, u, v, w], f2d_exact_solution)

Функции для работы программы

In [96]:
# Разные функции для работы программы
def x_i():
    array = np.zeros(N + 1)
    for i in range(N + 1):
        array[i] = float(a + i * h)
    return array


# Функция первоначальной задачи Коши
f1 = lambda input_x, input_u, input_v, input_w: input_v

f2 = lambda input_x, input_u, input_v, input_w: input_w

f3 = derivative2_exact_solution

# Метод Рунге Кутта 2 порядка:
def RungeKut(input_alpha, f1, f2, f3):
    input_u = np.zeros(N + 1)
    input_v = np.zeros(N + 1)
    input_w = np.zeros(N + 1)
    input_u[N] = B  # u
    input_v[N] = input_alpha  # v
    input_w[N] = C  # w
    k1 = np.zeros(3)
    k2 = np.zeros(3)

    for i in range(N, 1, -1):
        k1[0] = (h * f1(x_array[i], input_u[i], input_v[i], input_w[i]))
        k1[1] = (h * f2(x_array[i], input_u[i], input_v[i], input_w[i]))
        k1[2] = (h * f3(x_array[i], input_u[i], input_v[i], input_w[i]))

        k2[0] = (h * f1(x_array[i] + h, input_u[i] + k1[0], input_v[i] + k1[1], input_w[i] + k1[2]))
        k2[1] = (h * f2(x_array[i] + h, input_u[i] + k1[0], input_v[i] + k1[1], input_w[i] + k1[2]))
        k2[2] = (h * f3(x_array[i] + h, input_u[i] + k1[0], input_v[i] + k1[1], input_w[i] + k1[2]))

        input_u[i - 1] = input_u[i] + 0.5 * (k1[0] + k2[0])
        input_v[i - 1] = input_u[i] + 0.5 * (k1[1] + k2[1])
        input_w[i - 1] = input_w[i] + 0.5 * (k1[2] + k2[2])
    return input_u, input_v, input_w


def shooting_mehod():
    param_alpha0 = alpha0
    param_alpha1 = alpha1

    # du/dx = v
    f_u = f1
    # dv/dx = w
    f_v = f2
    # dw/dx
    f_w = exact_solution

    _, rk0_result, _ = RungeKut(param_alpha0, f_u, f_v, f_w)
    _, rk1_result, _ = RungeKut(param_alpha1, f_u, f_v, f_w)

    fi_alpha0 = rk0_result[0] - A
    fi_alpha1 = rk1_result[0] - A

    # Метод секущих для вычисления следующего приближения альфа
    param_alpha2 = param_alpha1 - fi_alpha1 * (param_alpha1 - param_alpha0) / (fi_alpha1 - fi_alpha0)

    _, rk2_result, _ = RungeKut(param_alpha2, f_u, f_v, f_w)
    fi = rk2_result[0] - A
    iterations_count = 1

    IER = 2
    while m.fabs(fi) > epsilon and iterations_count < K:
        iterations_count = iterations_count + 1
        param_alpha0 = param_alpha1
        param_alpha1 = param_alpha2
        fi_alpha0 = fi_alpha1
        fi_alpha1 = fi
        if m.fabs(fi_alpha1 - fi_alpha0) <= epsilon:
            return 2, None
        param_alpha2 = param_alpha1 * (param_alpha1 - param_alpha0) / (fi_alpha1 - fi_alpha0)
        _, rk_result, _ = RungeKut(param_alpha2, f_u, f_v, f_w)
        fi = rk_result[0] - A
        if iterations_count >= K:
            # Output code that iterations more than possible
            IER = 1
        if fi <= epsilon:
            # Output code that everything fine
            IER = 0

    return IER, param_alpha2

In [97]:
check = True
if N < 0 or K <= 0 or epsilon < 0:
    check = False
    IER = 2
    print('IER', IER, ':Неправильные значения параметров')
if check:
    x_array = x_i()

    IER, alpha = shooting_mehod()

    if IER == 0:
        th = ["x", "y", "deltaU", "y'", "deltaU'", "y''", "deltaU''"]
        td = []
        for i in np.arange(0, N + 1):
            td.append(
                round(x_array[i], 4)
            )
            td.append(
                round(exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]), 4)
            )
            td.append(
                round(abs(exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]) - u_array[i]), 4)
            )
            td.append(
                round(derivative_exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]), 4)
            )
            td.append(
                round(abs(derivative_exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]) - v_array[i]), 4)
            )
            td.append(
                round(derivative2_exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]), 4)
            )
            td.append(
                round(abs(derivative2_exact_solution(x_array[i], x_array[i], x_array[i], x_array[i]) - w_array[i]), 4)
            )
        columns = len(th)
        table = PrettyTable(th)
        td_data = td[:]
        while td_data:
            table.add_row(td_data[:columns])
            td_data = td_data[columns:]

        print('EIR = 0:Безошибочная работа программы')
        print('Количество итераций', iterations_count)
        print('Конечное значение параметра пристрелки', result_alpha)

        print(table)
    else:
        if IER == 1:
            print('EIR = 1: Количество итераций превысило установленный максимум')
            print(alpha)
        else:
            print('EIR = 2: Ошибка работы программы')

EIR = 2: Ошибка работы программы


C:\Users\splashgy\AppData\Local\Temp/ipykernel_8048/3590976442.py:60: RuntimeWarning: divide by zero encountered in double_scalars
  param_alpha2 = param_alpha1 - fi_alpha1 * (param_alpha1 - param_alpha0) / (fi_alpha1 - fi_alpha0)
